In [1]:
#coding:utf-8
import pandas as pd
import time
import gc
from sklearn.metrics import log_loss
def time2cov(time_):
    '''
    时间是根据天数推移，所以日期为脱敏，但是时间本身不脱敏
    :param time_: 
    :return: 
    '''
    return time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time_))
def pre_process(data):
    '''
    :param data: 
    :return: 
    '''

    print('预处理')
    print('item_category_list_ing')
    for i in range(3):
        data['category_%d'%(i)] = data['item_category_list'].apply(
            lambda x:x.split(";")[i] if len(x.split(";")) > i else " "
        )
    del data['item_category_list']

    print('item_property_list_ing')
    for i in range(3):
        data['property_%d'%(i)] = data['item_property_list'].apply(
            lambda x:x.split(";")[i] if len(x.split(";")) > i else " "
        )
    del data['item_property_list']

    print('context_timestamp_ing')
    data['context_timestamp'] = data['context_timestamp'].apply(time2cov)

    print('time')
    data['context_timestamp_tmp'] = pd.to_datetime(data['context_timestamp'])
    data['week'] = data['context_timestamp_tmp'].dt.weekday
    data['hour'] = data['context_timestamp_tmp'].dt.hour
    data['minute'] = data['context_timestamp_tmp'].dt.minute
    data['day'] = data['context_timestamp_tmp'].dt.day
    data['halfhour'] = (data['hour']+1)*3+(data['minute']/20).astype('int')
    del data['context_timestamp_tmp']
    del data['minute']
    del data['week']
    del data['hour']
    del data['predict_category_property']
    # del data['predict_property_1']
    # del data['predict_property_2']
    del data['shop_score_service']
    del data['shop_score_description']
    del data['shop_review_positive_rate']
    del data['shop_score_delivery']
    return data

In [2]:
print('train')
train = pd.read_table('train.txt',sep=' ',index_col=False)
dataa = pd.read_csv('train_all.csv',index_col=False)

train


## 特征选择

In [3]:
dataa.columns

Index(['instance_id', 'hour_num', 'item_click', 'shop_score_service_num',
       'shop_score_description_num', 'shop_review_positive_rate_num',
       'shop_score_delivery_num', 'user_click', 'property_0_click',
       'property_1_click', 'property_2_click', 'shop_click', 'brand_click',
       'category_1_pvnum', 'category_1_first', 'category_1_last',
       'property_0_pvnum', 'property_0_first', 'property_0_last',
       'property_1_pvnum', 'property_1_first', 'property_1_last',
       'item_id_pvnum', 'item_id_first', 'item_id_last', 'occupation_click',
       'user_gender_click', 'day_click'],
      dtype='object')

In [4]:
tezheng = ['instance_id', 'hour_num', 'item_click', 'shop_score_service_num',
       'shop_score_description_num', 'shop_review_positive_rate_num',
       'shop_score_delivery_num', 'user_click', 'property_0_click',
       'property_1_click', 'shop_click', 'brand_click',
       'category_1_pvnum', 'category_1_first', 'category_1_last',
       'property_0_pvnum', 'property_0_first', 'property_0_last',
       'property_1_pvnum', 'property_1_first', 'property_1_last',
       'item_id_pvnum', 'item_id_first', 'item_id_last', 'occupation_click',
       'user_gender_click', 'day_click']

In [5]:
dataa = dataa[tezheng]

In [6]:

train = pd.merge(train,dataa,on=['instance_id'],how='left')
del(dataa)
gc.collect()
train = pre_process(train)

预处理
item_category_list_ing
item_property_list_ing
context_timestamp_ing
time


In [7]:
all_data = train.copy()
print('all_shape',train.shape)
print(train['context_timestamp'].max())
val = train[train['day'] == 7]

# 空去 22 号

train = train[train['day'] != 7]

# train = train[train['context_timestamp']>'2018-09-19 23:59:59']
print(train.shape)
print(val.shape)


all_shape (10432036, 54)
2018-09-07 11:59:59
(9354861, 54)
(1077175, 54)


In [8]:

print('test')
test_a = pd.read_table('test.txt',sep=' ',index_col=False) 
data2a = pd.read_csv('test_all.csv',index_col=False) 

# del train[['user_id', 'category_1', 'property_2', 'property_0']]
# del val[['user_id', 'category_1', 'property_2', 'property_0']]
# del test_a[['user_id', 'category_1', 'property_2', 'property_0']]
# 这里是增加的内容


test


## 特征选择

In [9]:
data2a = data2a[tezheng]

In [10]:
test_a.columns

Index(['instance_id', 'item_id', 'item_category_list', 'item_property_list',
       'item_brand_id', 'item_city_id', 'item_price_level', 'item_sales_level',
       'item_collected_level', 'item_pv_level', 'user_id', 'user_gender_id',
       'user_age_level', 'user_occupation_id', 'user_star_level', 'context_id',
       'context_timestamp', 'context_page_id', 'predict_category_property',
       'shop_id', 'shop_review_num_level', 'shop_review_positive_rate',
       'shop_star_level', 'shop_score_service', 'shop_score_delivery',
       'shop_score_description'],
      dtype='object')

In [11]:
test_a = pd.merge(test_a,data2a,on=['instance_id'],how='left')
del(data2a)
gc.collect()
print(test_a.shape)
test_a = pre_process(test_a)
del train['item_city_id']
del val['item_city_id']
del test_a['item_city_id']
del train['shop_id']
del val['shop_id']
del test_a['shop_id']
del train['context_id']
del val['context_id']
del test_a['context_id']
del train['item_brand_id']
del val['item_brand_id']
del test_a['item_brand_id']
# del train['category_0']
# del val['category_0']
# del test_a['category_0']
# del train['category_2']
# del val['category_2']
# del test_a['category_2']
# del train['property_1']
# del val['property_1']
# del test_a['property_1']
# del train['property_2']
# del val['property_2']
# del test_a['property_2']

(519888, 52)
预处理
item_category_list_ing
item_property_list_ing
context_timestamp_ing
time


In [12]:

# import datetime
# def get_count_feat(all_data,data,long=3):
#     end_time = data['context_timestamp'].min()
#     begin_time = pd.to_datetime(end_time) - datetime.timedelta(days=long)
#     all_data['context_timestamp'] = pd.to_datetime(all_data['context_timestamp'])
#     all_data = all_data[
#         (all_data['context_timestamp']<end_time)&(all_data['context_timestamp']>=begin_time)
#                     ]
    
#     print(end_time)
#     print(begin_time)
#     print(all_data['context_timestamp'].max()-all_data['context_timestamp'].min())
#     item_count = all_data.groupby(['item_id'],as_index=False).size().reset_index()
#     item_count.rename(columns={0:'item_count'},inplace=True)

#     user_count = all_data.groupby(['user_id'], as_index=False).size().reset_index()
#     user_count.rename(columns={0: 'user_count'}, inplace=True)
#     return user_count,item_count




In [13]:

# train_user_count,train_item_count = get_count_feat(all_data,train,2)

# test_user_count,test_item_count = get_count_feat(all_data,test_a,2)

# val_user_count,val_item_count = get_count_feat(all_data,val,2)
# # print(train.columns)
# train = pd.merge(train,train_user_count,on=['user_id'],how='left')
# train = pd.merge(train,train_item_count,on=['item_id'],how='left')
# train = train.fillna(-1)
# val = pd.merge(val,val_user_count,on=['user_id'],how='left')
# val = pd.merge(val,val_item_count,on=['item_id'],how='left')
# val = val.fillna(-1)
# test_a = pd.merge(test_a,test_user_count,on=['user_id'],how='left')
# test_a = pd.merge(test_a,test_item_count,on=['item_id'],how='left')
# test_a = test_a.fillna(-1)

# 删掉上面增加的用户点击量与商品点击量
# del train['user_count']
# del train['item_count']
# del val['user_count']
# del val['item_count']
# del test_a['user_count']
# del test_a['item_count']

# user_item_count = pd.read_csv


y_train = train.pop('is_trade')
train_index = train.pop('instance_id')

y_val = val.pop('is_trade')
val_index = val.pop('instance_id')
test_index = test_a.pop('instance_id')

print(test_a.shape)
del train['context_timestamp']
del val['context_timestamp']
del test_a['context_timestamp']
del all_data


(519888, 48)


In [14]:
print('baseline ing ... ...')
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from scipy import sparse
from sklearn.linear_model import LogisticRegression

baseline ing ... ...


In [15]:

# print('baseline ing ... ...')
# from sklearn.preprocessing import LabelEncoder,OneHotEncoder
# from scipy import sparse
# from sklearn.linear_model import LogisticRegression
# # print(test_a.columns)

# enc = OneHotEncoder()
# lb = LabelEncoder()
# # feat_set = list(test_a.columns)

# del train['user_id']
# del test_a['user_id']
# del val['user_id']
# # del train['item_id']
# # del test_a['item_id']
# # del val['item_id']
# # 8288

# print(train.columns)

# # 自己使用onehot编码
# train['x_train'] = 1
# test_a['x_test'] = 1
# val['x_val'] = 1

# all_data = pd.concat([train,test_a,val])
# all_data = pd.get_dummies(all_data)
# print(all_data.shape)

# X_train = all_data.loc[all_data['x_train'].notnull()]
# X_test = all_data.loc[all_data['x_test'].notnull()]
# X_val = all_data.loc[all_data['x_val'].notnull()]

# del X_train['x_train']
# del X_train['x_test']
# del X_train['x_val']
# del X_test['x_train']
# del X_test['x_test']
# del X_test['x_val']
# del X_val['x_train']
# del X_val['x_test']
# del X_val['x_val']


In [16]:
# LogisticRegression?

In [17]:

# lr = LogisticRegression(C=5.0,penalty='l1')

# lr.fit(X_train, y_train)


In [18]:

# proba_val = lr.predict_proba(X_val)[:,1]
# proba_sub = lr.predict_proba(X_test)[:,1]
# print(log_loss(y_train,lr.predict_proba(X_train)[:,1]))
# print(log_loss(y_val,proba_val))
# print()

In [19]:
def change(x):
    if x == ' ':
        return -1
    else:
        return x

In [20]:
# val['category_0'] = val['category_0'].apply(change)
val['category_1'] = val['category_1'].apply(change)
val['category_2'] = val['category_2'].apply(change)
# val['property_0'] = val['property_0'].apply(change)
# val['property_1'] = val['property_1'].apply(change)
# val['property_2'] = val['property_2'].apply(change)
# train = pd.DataFrame(train,dtype='float')
# train['category_0'] = train['category_0'].apply(change)
train['category_1'] = train['category_1'].apply(change)
train['category_2'] = train['category_2'].apply(change)
# train['property_0'] = train['property_0'].apply(change)
# train['property_1'] = train['property_1'].apply(change)
# train['property_2'] = train['property_2'].apply(change)

# test_a['category_0'] = test_a['category_0'].apply(change)
test_a['category_1'] = test_a['category_1'].apply(change)
test_a['category_2'] = test_a['category_2'].apply(change)
# test_a['property_0'] = test_a['property_0'].apply(change)
# test_a['property_1'] = test_a['property_1'].apply(change)
# test_a['property_2'] = test_a['property_2'].apply(change)

In [21]:
# category_0, category_1, category_2, property_0, property_1, property_2
del val['category_0']
# del val['category_1']
# del val['category_2']
del val['property_0']
del val['property_1']
del val['property_2']
# train = pd.DataFrame(train,dtype='float')
del train['category_0']
# del train['category_1']
# del train['category_2']
del train['property_0']
del train['property_1']
del train['property_2']

del test_a['category_0']
# del test_a['category_1']
# del test_a['category_2']
del test_a['property_0']
del test_a['property_1']
del test_a['property_2']


# # df['col2'] = df['col2'].astype('int')
# # val['category_0'] = val['category_0'].astype('float64')
# # val['category_1'] = val['category_1'].astype('float64')
# # val['category_2'] = val['category_2'].astype('float64')
# # val['property_0'] = val['property_0'].astype('float64')
# # val['property_1'] = val['property_1'].astype('float64')
# # val['property_2'] = val['property_2'].astype('float64')
# # # train = pd.DataFrame(train,dtype='float')
# # train['category_0'] = train['category_0'].astype('float64')
# # train['category_1'] = train['category_1'].astype('float64')
# # train['category_2'] = train['category_2'].astype('float64')
# # train['property_0'] = train['property_0'].astype('float64')
# # train['property_1'] = train['property_1'].astype('float64')
# # train['property_2'] = train['property_2'].astype('float64')

# # test_a['category_0'] = test_a['category_0'].astype('float64')
# # test_a['category_1'] = test_a['category_1'].astype('float64')
# # test_a['category_2'] = test_a['category_2'].astype('float64')
# # test_a['property_0'] = test_a['property_0'].astype('float64')
# # test_a['property_1'] = test_a['property_1'].astype('float64')
# # test_a['property_2'] = test_a['property_2'].astype('float64')




In [22]:
# val['category_0'] = val['category_0'].astype('int64')
val['category_1'] = val['category_1'].astype('int64')
val['category_2'] = val['category_2'].astype('int64')
# val['property_0'] = val['property_0'].astype('int64')
# val['property_1'] = val['property_1'].astype('int64')
# val['property_2'] = val['property_2'].astype('int64')
# train = pd.DataFrame(train,dtype='float')
# train['category_0'] = train['category_0'].astype('int64')
train['category_1'] = train['category_1'].astype('int64')
train['category_2'] = train['category_2'].astype('int64')
# train['property_0'] = train['property_0'].astype('int64')
# train['property_1'] = train['property_1'].astype('int64')
# train['property_2'] = train['property_2'].astype('int64')

# test_a['category_0'] = test_a['category_0'].astype('int64')
test_a['category_1'] = test_a['category_1'].astype('int64')
test_a['category_2'] = test_a['category_2'].astype('int64')
# test_a['property_0'] = test_a['property_0'].astype('int64')
# test_a['property_1'] = test_a['property_1'].astype('int64')
# test_a['property_2'] = test_a['property_2'].astype('int64')

In [23]:
train.columns

Index(['item_id', 'item_price_level', 'item_sales_level',
       'item_collected_level', 'item_pv_level', 'user_id', 'user_gender_id',
       'user_age_level', 'user_occupation_id', 'user_star_level',
       'context_page_id', 'shop_review_num_level', 'shop_star_level',
       'hour_num', 'item_click', 'shop_score_service_num',
       'shop_score_description_num', 'shop_review_positive_rate_num',
       'shop_score_delivery_num', 'user_click', 'property_0_click',
       'property_1_click', 'shop_click', 'brand_click', 'category_1_pvnum',
       'category_1_first', 'category_1_last', 'property_0_pvnum',
       'property_0_first', 'property_0_last', 'property_1_pvnum',
       'property_1_first', 'property_1_last', 'item_id_pvnum', 'item_id_first',
       'item_id_last', 'occupation_click', 'user_gender_click', 'day_click',
       'category_1', 'category_2', 'day', 'halfhour'],
      dtype='object')

In [24]:
# def changeNaN(x):
#     if x==-1:
#         return float("nan") 
#     else:
#         return x

In [25]:
# train['item_price_level'] = train['item_price_level'].apply(changeNaN)
# train['item_sales_level'] = train['item_sales_level'].apply(changeNaN)
# train['item_collected_level'] = train['item_collected_level'].apply(changeNaN)
# train['item_pv_level'] = train['item_pv_level'].apply(changeNaN)
# train['user_gender_id'] = train['user_gender_id'].apply(changeNaN)
# train['user_age_level'] = train['user_age_level'].apply(changeNaN)
# train['user_occupation_id'] = train['user_occupation_id'].apply(changeNaN)
# train['shop_review_num_level'] = train['shop_review_num_level'].apply(changeNaN)
# train['shop_star_level'] = train['shop_star_level'].apply(changeNaN)
# pass

In [26]:
# val['item_price_level'] = val['item_price_level'].apply(changeNaN)
# val['item_sales_level'] = val['item_sales_level'].apply(changeNaN)
# val['item_collected_level'] = val['item_collected_level'].apply(changeNaN)
# val['item_pv_level'] = val['item_pv_level'].apply(changeNaN)
# val['user_gender_id'] = val['user_gender_id'].apply(changeNaN)
# val['user_age_level'] = val['user_age_level'].apply(changeNaN)
# val['user_occupation_id'] = val['user_occupation_id'].apply(changeNaN)
# val['shop_review_num_level'] = val['shop_review_num_level'].apply(changeNaN)
# val['shop_star_level'] = val['shop_star_level'].apply(changeNaN)
# pass

In [27]:
# test_a['item_price_level'] = test_a['item_price_level'].apply(changeNaN)
# test_a['item_sales_level'] = test_a['item_sales_level'].apply(changeNaN)
# test_a['item_collected_level'] = test_a['item_collected_level'].apply(changeNaN)
# test_a['item_pv_level'] = test_a['item_pv_level'].apply(changeNaN)
# test_a['user_gender_id'] = test_a['user_gender_id'].apply(changeNaN)
# test_a['user_age_level'] = test_a['user_age_level'].apply(changeNaN)
# test_a['user_occupation_id'] = test_a['user_occupation_id'].apply(changeNaN)
# test_a['shop_review_num_level'] = test_a['shop_review_num_level'].apply(changeNaN)
# test_a['shop_star_level'] = test_a['shop_star_level'].apply(changeNaN)
# pass

In [28]:
# train[['item_id','user_gender_id']].groupby('user_gender_id').count()

In [29]:
# train.count()

In [30]:

import lightgbm as lgb
gbm = lgb.LGBMRegressor(objective='binary',
                        num_leaves = 128,
#                         max_depth = 8,
                        device = "gpu",
                        gpu_platform_id = 0,
                        gpu_device_id = 0,
                        learning_rate=0.01,
                        n_estimators=2000,
                        colsample_bytree = 0.65,
                        subsample = 0.7,
                        reg_alpha = 1,
                       )
gbm.fit(train, y_train,
        categorical_feature  = [ "category_1", "category_2"],
        eval_set=[(val, y_val)],
        eval_metric='binary_logloss',
        early_stopping_rounds=150)



[1]	valid_0's binary_logloss: 0.684326
Training until validation scores don't improve for 150 rounds.
[2]	valid_0's binary_logloss: 0.675604
[3]	valid_0's binary_logloss: 0.666653
[4]	valid_0's binary_logloss: 0.657699
[5]	valid_0's binary_logloss: 0.649528
[6]	valid_0's binary_logloss: 0.641239
[7]	valid_0's binary_logloss: 0.633319
[8]	valid_0's binary_logloss: 0.625751
[9]	valid_0's binary_logloss: 0.618003
[10]	valid_0's binary_logloss: 0.612083
[11]	valid_0's binary_logloss: 0.603614
[12]	valid_0's binary_logloss: 0.596535
[13]	valid_0's binary_logloss: 0.5896
[14]	valid_0's binary_logloss: 0.582817
[15]	valid_0's binary_logloss: 0.576167
[16]	valid_0's binary_logloss: 0.570405
[17]	valid_0's binary_logloss: 0.563979
[18]	valid_0's binary_logloss: 0.557528
[19]	valid_0's binary_logloss: 0.551359
[20]	valid_0's binary_logloss: 0.545272
[21]	valid_0's binary_logloss: 0.539302
[22]	valid_0's binary_logloss: 0.533566
[23]	valid_0's binary_logloss: 0.527816
[24]	valid_0's binary_loglos

[203]	valid_0's binary_logloss: 0.192596
[204]	valid_0's binary_logloss: 0.192305
[205]	valid_0's binary_logloss: 0.192016
[206]	valid_0's binary_logloss: 0.19174
[207]	valid_0's binary_logloss: 0.191465
[208]	valid_0's binary_logloss: 0.191203
[209]	valid_0's binary_logloss: 0.190947
[210]	valid_0's binary_logloss: 0.190683
[211]	valid_0's binary_logloss: 0.190436
[212]	valid_0's binary_logloss: 0.190193
[213]	valid_0's binary_logloss: 0.18996
[214]	valid_0's binary_logloss: 0.189722
[215]	valid_0's binary_logloss: 0.189493
[216]	valid_0's binary_logloss: 0.189268
[217]	valid_0's binary_logloss: 0.189044
[218]	valid_0's binary_logloss: 0.188844
[219]	valid_0's binary_logloss: 0.188636
[220]	valid_0's binary_logloss: 0.18843
[221]	valid_0's binary_logloss: 0.188234
[222]	valid_0's binary_logloss: 0.188036
[223]	valid_0's binary_logloss: 0.187852
[224]	valid_0's binary_logloss: 0.187667
[225]	valid_0's binary_logloss: 0.187484
[226]	valid_0's binary_logloss: 0.187299
[227]	valid_0's bin

[404]	valid_0's binary_logloss: 0.192387
[405]	valid_0's binary_logloss: 0.192509
[406]	valid_0's binary_logloss: 0.19262
[407]	valid_0's binary_logloss: 0.192784
[408]	valid_0's binary_logloss: 0.19289
[409]	valid_0's binary_logloss: 0.193003
[410]	valid_0's binary_logloss: 0.193066
[411]	valid_0's binary_logloss: 0.193145
[412]	valid_0's binary_logloss: 0.193294
[413]	valid_0's binary_logloss: 0.193386
[414]	valid_0's binary_logloss: 0.193455
[415]	valid_0's binary_logloss: 0.193574
[416]	valid_0's binary_logloss: 0.19366
[417]	valid_0's binary_logloss: 0.193786
[418]	valid_0's binary_logloss: 0.193887
[419]	valid_0's binary_logloss: 0.193982
[420]	valid_0's binary_logloss: 0.194117
[421]	valid_0's binary_logloss: 0.194196
[422]	valid_0's binary_logloss: 0.194293
[423]	valid_0's binary_logloss: 0.194405
[424]	valid_0's binary_logloss: 0.194525
[425]	valid_0's binary_logloss: 0.194596
[426]	valid_0's binary_logloss: 0.194637
[427]	valid_0's binary_logloss: 0.194733
[428]	valid_0's bin

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=0.65,
       device='gpu', gpu_device_id=0, gpu_platform_id=0,
       learning_rate=0.01, max_depth=-1, min_child_samples=20,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=2000,
       n_jobs=-1, num_leaves=128, objective='binary', random_state=None,
       reg_alpha=1, reg_lambda=0.0, silent=True, subsample=0.7,
       subsample_for_bin=200000, subsample_freq=1)

In [31]:
print('Start predicting...')
# predict
# y_pred_1 = gbm.predict(val, num_iteration=gbm.best_iteration_ )
y_sub_1 = gbm.predict(test_a, num_iteration=gbm.best_iteration_ )
print(log_loss(y_train,gbm.predict(train)))
# print(log_loss(y_val,y_pred_1))

# print(log_loss(y_val,(y_pred_1 + proba_val)/2))#0.0802392979662

Start predicting...
0.0734206514259


In [33]:

# import numpy as np
# bes = 1
# for i in np.arange(0,1,0.01):
#     if(log_loss(y_val,(i * y_pred_1 + (1-i) * proba_val)) < bes):
#         bes = log_loss(y_val,(y_pred_1 + proba_val)/2)
# print(bes)#0.083131236937

# xx_analyse = pd.DataFrame()
# xx_analyse['ture'] = list(y_val)
# xx_analyse['pre'] = list(proba_val)
# xx_analyse['pre_1'] = list(y_pred_1)
# xx_analyse.to_csv('temp.csv',index=False)
#
# sub = pd.DataFrame()
# sub['instance_id'] = list(test_index)
# sub['predicted_score'] = list(y_sub_1)
# sub.to_csv('feather/20180327.txt',sep=" ",index=False)

In [34]:
y_sub_1.shape

(519888,)

In [35]:
y_sub_1

array([ 0.03500097,  0.0337831 ,  0.04590906, ...,  0.03276625,
        0.05857461,  0.04220761])

In [36]:
sub = pd.DataFrame()
sub['instance_id'] = list(test_index)
sub['predicted_score'] = list(y_sub_1)
sub.to_csv('20180430.txt',sep=" ",index=False)